# DOS Reform Bills 
scraped from congress.gov 1951-2024 (limited data for 1950-1972)

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [49]:
%%capture output 

# RAW
data = pd.read_csv('rawdata.csv').iloc[:,0:17]

In [50]:
data = data.drop(data[data['Legislation Number'].isna()].index, axis=0)
data = data.fillna('-')
data.head()

,Legislation Number,URL,Congress,Title,Amends Bill,Sponsor,Date Offered,Date of Introduction,Number of Cosponsors,Date Submitted,Date Proposed,Committees,Latest Action,Latest Action Date,Latest Summary,Subject,allsubjects
0,H.Res. 723,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Providing for consideration of the bill (H.R. ...,-,"Roy, Chip [Rep.-R-TX-21]",-,9/26/23,0.0,-,-,House - Rules,Motion to reconsider laid on the table Agreed ...,9/26/23,Sets forth the rule for consideration of the b...,Congress,"Congress, House of Representatives, Legislativ..."
1,H.R. 4665,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Diaz-Balart, Mario [Rep.-R-FL-26]",-,7/17/23,0.0,-,-,House - Appropriations,Received in the Senate. Read twice. Placed on ...,10/3/23,"<p><b>Department of State, Foreign Operations...",International Affairs,"International Affairs, Africa, China, Executiv..."
2,H.Res. 433,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Recognizing the historic significance of the 3...,-,"Kelly, Trent [Rep.-R-MS-1]",-,5/22/23,1.0,-,-,House - Armed Services,Referred to the House Committee on Armed Servi...,5/22/23,<p>This resolution recognizes the 30th annive...,International Affairs,International Affairs
3,S. 2438,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Coons, Christopher A. [Sen.-D-DE]",-,7/20/23,0.0,-,-,Senate - Appropriations,Placed on Senate Legislative Calendar under Ge...,7/20/23,"<p><b>Department of State, Foreign Operations...",International Affairs,"International Affairs, Peace Corps, Sex offens..."
4,H.R. 1159,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To amend the Taiwan Assurance Act of 2020 to r...,-,"Wagner, Ann [Rep.-R-MO-2]",-,2/24/23,6.0,-,-,House - Foreign Affairs | Senate - Foreign Rel...,Received in the Senate and Read twice and refe...,3/23/23,<p>This bill modifies an existing requirement...,International Affairs,"International Affairs, Taiwan, Sovereignty, re..."


# Pre cleaning

## Remove private laws 'for the relief of'

In [52]:
data['private'] = data.Title.dropna().str.match(r'For the relief of.*')
data = data[data.private==False]
data = data.drop('private', axis=1)
data.head()

,Legislation Number,URL,Congress,Title,Amends Bill,Sponsor,Date Offered,Date of Introduction,Number of Cosponsors,Date Submitted,Date Proposed,Committees,Latest Action,Latest Action Date,Latest Summary,Subject,allsubjects
0,H.Res. 723,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Providing for consideration of the bill (H.R. ...,-,"Roy, Chip [Rep.-R-TX-21]",-,9/26/23,0.0,-,-,House - Rules,Motion to reconsider laid on the table Agreed ...,9/26/23,Sets forth the rule for consideration of the b...,Congress,"Congress, House of Representatives, Legislativ..."
1,H.R. 4665,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Diaz-Balart, Mario [Rep.-R-FL-26]",-,7/17/23,0.0,-,-,House - Appropriations,Received in the Senate. Read twice. Placed on ...,10/3/23,"<p><b>Department of State, Foreign Operations...",International Affairs,"International Affairs, Africa, China, Executiv..."
2,H.Res. 433,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Recognizing the historic significance of the 3...,-,"Kelly, Trent [Rep.-R-MS-1]",-,5/22/23,1.0,-,-,House - Armed Services,Referred to the House Committee on Armed Servi...,5/22/23,<p>This resolution recognizes the 30th annive...,International Affairs,International Affairs
3,S. 2438,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Coons, Christopher A. [Sen.-D-DE]",-,7/20/23,0.0,-,-,Senate - Appropriations,Placed on Senate Legislative Calendar under Ge...,7/20/23,"<p><b>Department of State, Foreign Operations...",International Affairs,"International Affairs, Peace Corps, Sex offens..."
4,H.R. 1159,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To amend the Taiwan Assurance Act of 2020 to r...,-,"Wagner, Ann [Rep.-R-MO-2]",-,2/24/23,6.0,-,-,House - Foreign Affairs | Senate - Foreign Rel...,Received in the Senate and Read twice and refe...,3/23/23,<p>This bill modifies an existing requirement...,International Affairs,"International Affairs, Taiwan, Sovereignty, re..."


## Fix HTML summary

In [53]:
from bs4 import BeautifulSoup

text = []
for html in data['Latest Summary']:
    soup = BeautifulSoup(html, "html.parser")
    t = soup.get_text(strip=True, separator=" ")
    text += [t]
    
data['summary'] = text

In [54]:
data = data.drop(columns=['Latest Summary'])

In [55]:
data.head()

,Legislation Number,URL,Congress,Title,Amends Bill,Sponsor,Date Offered,Date of Introduction,Number of Cosponsors,Date Submitted,Date Proposed,Committees,Latest Action,Latest Action Date,Subject,allsubjects,summary
0,H.Res. 723,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Providing for consideration of the bill (H.R. ...,-,"Roy, Chip [Rep.-R-TX-21]",-,9/26/23,0.0,-,-,House - Rules,Motion to reconsider laid on the table Agreed ...,9/26/23,Congress,"Congress, House of Representatives, Legislativ...",Sets forth the rule for consideration of the b...
1,H.R. 4665,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Diaz-Balart, Mario [Rep.-R-FL-26]",-,7/17/23,0.0,-,-,House - Appropriations,Received in the Senate. Read twice. Placed on ...,10/3/23,International Affairs,"International Affairs, Africa, China, Executiv...","Department of State, Foreign Operations, and R..."
2,H.Res. 433,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Recognizing the historic significance of the 3...,-,"Kelly, Trent [Rep.-R-MS-1]",-,5/22/23,1.0,-,-,House - Armed Services,Referred to the House Committee on Armed Servi...,5/22/23,International Affairs,International Affairs,This resolution recognizes the 30th anniversar...
3,S. 2438,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Coons, Christopher A. [Sen.-D-DE]",-,7/20/23,0.0,-,-,Senate - Appropriations,Placed on Senate Legislative Calendar under Ge...,7/20/23,International Affairs,"International Affairs, Peace Corps, Sex offens...","Department of State, Foreign Operations, and R..."
4,H.R. 1159,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To amend the Taiwan Assurance Act of 2020 to r...,-,"Wagner, Ann [Rep.-R-MO-2]",-,2/24/23,6.0,-,-,House - Foreign Affairs | Senate - Foreign Rel...,Received in the Senate and Read twice and refe...,3/23/23,International Affairs,"International Affairs, Taiwan, Sovereignty, re...",This bill modifies an existing requirement for...


## Parse out Congress nums

In [59]:
# Extract congress number
data['congress_num'] = data.Congress.str.extract(r'(\d*)[a-z]* Congress').astype(str)

# Prefix (House/Senate)
data['bill_type'] = [string[0] for string in  data['Legislation Number'].str.split(' ')]

# Leglislation number
data['bill_num'] = [string[1] for string in  data['Legislation Number'].str.split(' ')]

for i in data.bill_type:
    if i in ['H.R.', 'H.Res.', 'H.Con.Res', 'H.J.Res.','H.Amdt']:
        data['house'] = 'HR'
    else:
        data['house'] = 'S'

data.head()

,Legislation Number,URL,Congress,Title,Amends Bill,Sponsor,Date Offered,Date of Introduction,Number of Cosponsors,Date Submitted,...,Committees,Latest Action,Latest Action Date,Subject,allsubjects,summary,congress_num,bill_type,bill_num,house
0,H.Res. 723,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Providing for consideration of the bill (H.R. ...,-,"Roy, Chip [Rep.-R-TX-21]",-,9/26/23,0.0,-,...,House - Rules,Motion to reconsider laid on the table Agreed ...,9/26/23,Congress,"Congress, House of Representatives, Legislativ...",Sets forth the rule for consideration of the b...,118,H.Res.,723,HR
1,H.R. 4665,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Diaz-Balart, Mario [Rep.-R-FL-26]",-,7/17/23,0.0,-,...,House - Appropriations,Received in the Senate. Read twice. Placed on ...,10/3/23,International Affairs,"International Affairs, Africa, China, Executiv...","Department of State, Foreign Operations, and R...",118,H.R.,4665,HR
2,H.Res. 433,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Recognizing the historic significance of the 3...,-,"Kelly, Trent [Rep.-R-MS-1]",-,5/22/23,1.0,-,...,House - Armed Services,Referred to the House Committee on Armed Servi...,5/22/23,International Affairs,International Affairs,This resolution recognizes the 30th anniversar...,118,H.Res.,433,HR
3,S. 2438,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Coons, Christopher A. [Sen.-D-DE]",-,7/20/23,0.0,-,...,Senate - Appropriations,Placed on Senate Legislative Calendar under Ge...,7/20/23,International Affairs,"International Affairs, Peace Corps, Sex offens...","Department of State, Foreign Operations, and R...",118,S.,2438,HR
4,H.R. 1159,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To amend the Taiwan Assurance Act of 2020 to r...,-,"Wagner, Ann [Rep.-R-MO-2]",-,2/24/23,6.0,-,...,House - Foreign Affairs | Senate - Foreign Rel...,Received in the Senate and Read twice and refe...,3/23/23,International Affairs,"International Affairs, Taiwan, Sovereignty, re...",This bill modifies an existing requirement for...,118,H.R.,1159,HR


In [58]:
data.bill_type.value_counts()

H.R.          6148
S.            4245
H.Res.         928
S.Res.         605
S.Amdt.        544
H.Con.Res.     330
H.J.Res.       259
H.Amdt.        166
S.Con.Res.     111
S.J.Res.        86
S.Up.Amdt.       3
Name: bill_type, dtype: int64

In [60]:
data.head()

,Legislation Number,URL,Congress,Title,Amends Bill,Sponsor,Date Offered,Date of Introduction,Number of Cosponsors,Date Submitted,...,Committees,Latest Action,Latest Action Date,Subject,allsubjects,summary,congress_num,bill_type,bill_num,house
0,H.Res. 723,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Providing for consideration of the bill (H.R. ...,-,"Roy, Chip [Rep.-R-TX-21]",-,9/26/23,0.0,-,...,House - Rules,Motion to reconsider laid on the table Agreed ...,9/26/23,Congress,"Congress, House of Representatives, Legislativ...",Sets forth the rule for consideration of the b...,118,H.Res.,723,HR
1,H.R. 4665,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Diaz-Balart, Mario [Rep.-R-FL-26]",-,7/17/23,0.0,-,...,House - Appropriations,Received in the Senate. Read twice. Placed on ...,10/3/23,International Affairs,"International Affairs, Africa, China, Executiv...","Department of State, Foreign Operations, and R...",118,H.R.,4665,HR
2,H.Res. 433,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Recognizing the historic significance of the 3...,-,"Kelly, Trent [Rep.-R-MS-1]",-,5/22/23,1.0,-,...,House - Armed Services,Referred to the House Committee on Armed Servi...,5/22/23,International Affairs,International Affairs,This resolution recognizes the 30th anniversar...,118,H.Res.,433,HR
3,S. 2438,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Coons, Christopher A. [Sen.-D-DE]",-,7/20/23,0.0,-,...,Senate - Appropriations,Placed on Senate Legislative Calendar under Ge...,7/20/23,International Affairs,"International Affairs, Peace Corps, Sex offens...","Department of State, Foreign Operations, and R...",118,S.,2438,HR
4,H.R. 1159,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To amend the Taiwan Assurance Act of 2020 to r...,-,"Wagner, Ann [Rep.-R-MO-2]",-,2/24/23,6.0,-,...,House - Foreign Affairs | Senate - Foreign Rel...,Received in the Senate and Read twice and refe...,3/23/23,International Affairs,"International Affairs, Taiwan, Sovereignty, re...",This bill modifies an existing requirement for...,118,H.R.,1159,HR


## Passed column

In [61]:
data['Latest Action']

0        Motion to reconsider laid on the table Agreed ...
1        Received in the Senate. Read twice. Placed on ...
2        Referred to the House Committee on Armed Servi...
3        Placed on Senate Legislative Calendar under Ge...
4        Received in the Senate and Read twice and refe...
                               ...                        
14059                         Became Public Law No. 82-376
14060                          Became Public Law No. 82-50
14061                          Became Public Law No. 82-49
14062                         Became Public Law No. 82-213
14063                         Became Public Law No. 82-212
Name: Latest Action, Length: 13425, dtype: object

In [62]:
data['passed'] = data['Latest Action'].str.contains(r'(?i).*law.*')
data.head()

,Legislation Number,URL,Congress,Title,Amends Bill,Sponsor,Date Offered,Date of Introduction,Number of Cosponsors,Date Submitted,...,Latest Action,Latest Action Date,Subject,allsubjects,summary,congress_num,bill_type,bill_num,house,passed
0,H.Res. 723,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Providing for consideration of the bill (H.R. ...,-,"Roy, Chip [Rep.-R-TX-21]",-,9/26/23,0.0,-,...,Motion to reconsider laid on the table Agreed ...,9/26/23,Congress,"Congress, House of Representatives, Legislativ...",Sets forth the rule for consideration of the b...,118,H.Res.,723,HR,False
1,H.R. 4665,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Diaz-Balart, Mario [Rep.-R-FL-26]",-,7/17/23,0.0,-,...,Received in the Senate. Read twice. Placed on ...,10/3/23,International Affairs,"International Affairs, Africa, China, Executiv...","Department of State, Foreign Operations, and R...",118,H.R.,4665,HR,False
2,H.Res. 433,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Recognizing the historic significance of the 3...,-,"Kelly, Trent [Rep.-R-MS-1]",-,5/22/23,1.0,-,...,Referred to the House Committee on Armed Servi...,5/22/23,International Affairs,International Affairs,This resolution recognizes the 30th anniversar...,118,H.Res.,433,HR,False
3,S. 2438,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Coons, Christopher A. [Sen.-D-DE]",-,7/20/23,0.0,-,...,Placed on Senate Legislative Calendar under Ge...,7/20/23,International Affairs,"International Affairs, Peace Corps, Sex offens...","Department of State, Foreign Operations, and R...",118,S.,2438,HR,False
4,H.R. 1159,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To amend the Taiwan Assurance Act of 2020 to r...,-,"Wagner, Ann [Rep.-R-MO-2]",-,2/24/23,6.0,-,...,Received in the Senate and Read twice and refe...,3/23/23,International Affairs,"International Affairs, Taiwan, Sovereignty, re...",This bill modifies an existing requirement for...,118,H.R.,1159,HR,False


In [64]:
data['passed'].value_counts()

False    10864
True      2561
Name: passed, dtype: int64

# Filtering

Hierarchy of filtering

1. State Department / Department of State / Secretary of State
2. Check for tags

- Financial: 'appropriat', 'authoriz', 'fiscal','fund', 'spend', 
- Reform: 'modern', 'improv', 'reform', 
- Workforce:  'diplom', 'personnel', 'train', 'diversity', 'inclusion', 'foreign service'
- Other: 'statecraft', 'science', 'tech'

## Combine Title + Summary

In [65]:
data['text'] = [i + " " + j for i, j in zip(data.Title, data.summary)]

In [66]:
data.head()

,Legislation Number,URL,Congress,Title,Amends Bill,Sponsor,Date Offered,Date of Introduction,Number of Cosponsors,Date Submitted,...,Latest Action Date,Subject,allsubjects,summary,congress_num,bill_type,bill_num,house,passed,text
0,H.Res. 723,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Providing for consideration of the bill (H.R. ...,-,"Roy, Chip [Rep.-R-TX-21]",-,9/26/23,0.0,-,...,9/26/23,Congress,"Congress, House of Representatives, Legislativ...",Sets forth the rule for consideration of the b...,118,H.Res.,723,HR,False,Providing for consideration of the bill (H.R. ...
1,H.R. 4665,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Diaz-Balart, Mario [Rep.-R-FL-26]",-,7/17/23,0.0,-,...,10/3/23,International Affairs,"International Affairs, Africa, China, Executiv...","Department of State, Foreign Operations, and R...",118,H.R.,4665,HR,False,"Department of State, Foreign Operations, and R..."
2,H.Res. 433,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Recognizing the historic significance of the 3...,-,"Kelly, Trent [Rep.-R-MS-1]",-,5/22/23,1.0,-,...,5/22/23,International Affairs,International Affairs,This resolution recognizes the 30th anniversar...,118,H.Res.,433,HR,False,Recognizing the historic significance of the 3...
3,S. 2438,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Coons, Christopher A. [Sen.-D-DE]",-,7/20/23,0.0,-,...,7/20/23,International Affairs,"International Affairs, Peace Corps, Sex offens...","Department of State, Foreign Operations, and R...",118,S.,2438,HR,False,"Department of State, Foreign Operations, and R..."
4,H.R. 1159,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To amend the Taiwan Assurance Act of 2020 to r...,-,"Wagner, Ann [Rep.-R-MO-2]",-,2/24/23,6.0,-,...,3/23/23,International Affairs,"International Affairs, Taiwan, Sovereignty, re...",This bill modifies an existing requirement for...,118,H.R.,1159,HR,False,To amend the Taiwan Assurance Act of 2020 to r...


## Cross reference bills from Office of Historian

In [67]:
data2 = pd.read_csv('officehistorian.csv')

# House/Senate
data2['bill_type'] = [string[0] for string in  data2['Legislation Number'].str.split(' ')]

# Leglislation number
data2['bill_num'] = [string[1] for string in  data2['Legislation Number'].str.split(' ')]

for i in data2.bill_type:
    if i in ['H.R.', 'H.Res.', 'H.Con.Res', 'H.J.Res.','H.Amdt']:
        data2['house'] = 'HR'
    else:
        data2['house'] = 'S'
        
# Congress number
data2['congress_num'] = data2.URL.str.extract(r'.*gov\/bill\/(\d*).*').astype(str)

data2['officehistorian'] = True

In [68]:
data2.shape

(275, 11)

In [70]:
data = data.merge(data2[['Legislation Number', 'officehistorian', 'congress_num']],
           how = 'left',
           left_on=['Legislation Number', 'congress_num'],
           right_on=['Legislation Number', 'congress_num'])
data.officehistorian.fillna('FALSE')

0        FALSE
1        FALSE
2        FALSE
3        FALSE
4        FALSE
         ...  
13420    FALSE
13421    FALSE
13422    FALSE
13423    FALSE
13424    FALSE
Name: officehistorian, Length: 13425, dtype: object

In [72]:
sum(data.officehistorian==True)

265

## DOS in text

In [73]:
sum(data.text.str.contains(r'(?i)(of state)|(state department)'))

/var/folders/js/7x9j4wbs6nj983t_k4_l9x5r0000gn/T/ipykernel_62058/2363036543.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sum(data.text.str.contains(r'(?i)(of state)|(state department)'))


2496

In [74]:
data['DOS_in_text'] = data.text.str.contains(r'(?i)(of state)|(state department)')

/var/folders/js/7x9j4wbs6nj983t_k4_l9x5r0000gn/T/ipykernel_62058/1418497105.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['DOS_in_text'] = data.text.str.contains(r'(?i)(of state)|(state department)')


## Contains keywords

- Financial: 'appropriat', 'authoriz', 'fiscal','fund', 'spend', 
- Reform: 'modern', 'improv', 'reform', 
- Workforce:  'diplom', 'personnel', 'train', 'diversity', 'inclusion', 'foreign service'
- Other: 'statecraft', 'science', 'tech'

In [75]:
data['tag_financial'] = data.text.str.contains(r'(?i)(appropriat)|(authoriz)|(fiscal)|(spend)|(fund)')
data['tag_reform'] = data.text.str.contains(r'(?i)(modern)|(improv)|(reform)')
data['tag_workforce'] = data.text.str.contains(r'(?i)(diplom)|(personnel)|(train)|(diversity)|(inclusion)|(foreign service)')
data['tag_foreign'] = data.text.str.contains(r'(?i)(foreign affair)|(international affair)|(foreign relation)|(international relation)')
data['tag_aid'] = data.text.str.contains(r'(?i)(foreign aid)|(usaid)')
data['tag_dod'] = data.text.str.contains(r'(?i)(department of defense)|(\sdod\s)|(national defense)')
data['tag_other'] = data.text.str.contains(r'(?i)(statecraft)|(science)|(tech)')

/var/folders/js/7x9j4wbs6nj983t_k4_l9x5r0000gn/T/ipykernel_62058/4003290879.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['tag_financial'] = data.text.str.contains(r'(?i)(appropriat)|(authoriz)|(fiscal)|(spend)|(fund)')
/var/folders/js/7x9j4wbs6nj983t_k4_l9x5r0000gn/T/ipykernel_62058/4003290879.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['tag_reform'] = data.text.str.contains(r'(?i)(modern)|(improv)|(reform)')
/var/folders/js/7x9j4wbs6nj983t_k4_l9x5r0000gn/T/ipykernel_62058/4003290879.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data['tag_workforce'] = data.text.str.contains(r'(?i)(diplom)|(personnel)|(train)|(diversity)|(inclusion)|(foreign service)')
/var/folders/js/7x9j4wbs6nj983t_k4_l9x5r0

In [76]:
data.head()

,Legislation Number,URL,Congress,Title,Amends Bill,Sponsor,Date Offered,Date of Introduction,Number of Cosponsors,Date Submitted,...,text,officehistorian,DOS_in_text,tag_financial,tag_reform,tag_workforce,tag_foreign,tag_aid,tag_dod,tag_other
0,H.Res. 723,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Providing for consideration of the bill (H.R. ...,-,"Roy, Chip [Rep.-R-TX-21]",-,9/26/23,0.0,-,...,Providing for consideration of the bill (H.R. ...,NaN,True,True,False,False,False,False,True,False
1,H.R. 4665,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Diaz-Balart, Mario [Rep.-R-FL-26]",-,7/17/23,0.0,-,...,"Department of State, Foreign Operations, and R...",NaN,True,True,False,False,True,True,False,False
2,H.Res. 433,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Recognizing the historic significance of the 3...,-,"Kelly, Trent [Rep.-R-MS-1]",-,5/22/23,1.0,-,...,Recognizing the historic significance of the 3...,NaN,False,False,False,False,False,False,True,False
3,S. 2438,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),"Department of State, Foreign Operations, and R...",-,"Coons, Christopher A. [Sen.-D-DE]",-,7/20/23,0.0,-,...,"Department of State, Foreign Operations, and R...",NaN,True,True,True,False,True,True,False,False
4,H.R. 1159,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),To amend the Taiwan Assurance Act of 2020 to r...,-,"Wagner, Ann [Rep.-R-MO-2]",-,2/24/23,6.0,-,...,To amend the Taiwan Assurance Act of 2020 to r...,NaN,True,False,False,True,False,False,False,False


In [78]:
data = data[(data.officehistorian==True)|(data.DOS_in_text==True)
             |(data.tag_financial==True)|(data.tag_other==True)
             |(data.tag_reform==True)|(data.tag_workforce==True)
             |(data.tag_foreign==True)|(data.tag_aid==True)]

In [79]:
data.shape

(5519, 32)

In [80]:
data.to_csv('tags.csv')

## Hand-code

Create a `relevant` column in `tagged.csv` and hand-code 1 for relevant

In [82]:
data = pd.read_csv('tags.csv')

In [83]:
relevant = data[data['relevant']==1]

In [84]:
relevant = relevant[['Legislation Number', 'Title',
            'congress_num', 'bill_type', 'bill_num',
            'URL', 'Date of Introduction', 'Latest Action Date',
            'Sponsor', 'Number of Cosponsors',
            'Committees', 'allsubjects',
            'summary', 'passed?',
            'DOS_in_text', 'relevant_sophia', 'tag_reform', 'tag_financial',
            'financial type',
            'tag_workforce', 'tag_foreign', 'tag_aid', 'tag_dod', 'tag_other']]

In [85]:
# remove all non-DOS authorization bills
dos = relevant[(relevant['financial type']=='DOS')|(relevant['tag_financial']==False)]

In [86]:
dos.to_csv('dos.csv')

# Post cleaning

## Collapse duplicate bills

In [87]:
dos = pd.read_csv('dos.csv', index_col=0)

In [88]:
dos.head()

,Legislation Number,Title,congress_num,bill_type,bill_num,URL,Date of Introduction,Latest Action Date,Sponsor,Number of Cosponsors,...,DOS_in_text,relevant_sophia,tag_reform,tag_financial,financial type,tag_workforce,tag_foreign,tag_aid,tag_dod,tag_other
0,H.Res. 723,Providing for consideration of the bill (H.R. ...,118.0,H.Res.,723.0,https://www.congress.gov/bill/118th-congress/h...,9/26/23,9/26/23,"Roy, Chip [Rep.-R-TX-21]",0.0,...,True,False,False,True,DOS,False,False,False,True,False
1,H.R. 4665,"Department of State, Foreign Operations, and R...",118.0,H.R.,4665.0,https://www.congress.gov/bill/118th-congress/h...,7/17/23,10/3/23,"Diaz-Balart, Mario [Rep.-R-FL-26]",0.0,...,True,False,False,True,DOS,False,True,True,False,False
2,S. 2438,"Department of State, Foreign Operations, and R...",118.0,S.,2438.0,https://www.congress.gov/bill/118th-congress/s...,7/20/23,7/20/23,"Coons, Christopher A. [Sen.-D-DE]",0.0,...,True,False,True,True,DOS,False,True,True,False,False
4,H.R. 2300,To provide for a limitation on availability of...,118.0,H.R.,2300.0,https://www.congress.gov/bill/118th-congress/h...,3/29/23,3/29/23,"Biggs, Andy [Rep.-R-AZ-5]",4.0,...,True,False,False,True,DOS,False,True,False,False,False
6,S. 2043,Department of State Authorization Act of 2023,118.0,S.,2043.0,https://www.congress.gov/bill/118th-congress/s...,6/15/23,8/22/23,"Menendez, Robert [Sen.-D-NJ]",0.0,...,True,False,False,True,DOS,False,False,False,False,False


In [89]:
dos.columns

Index(['Legislation Number', 'Title', 'congress_num', 'bill_type', 'bill_num',
       'URL', 'Date of Introduction', 'Latest Action Date', 'Sponsor',
       'Number of Cosponsors', 'Committees', 'allsubjects', 'summary',
       'passed?', 'DOS_in_text', 'relevant_sophia', 'tag_reform',
       'tag_financial', 'financial type', 'tag_workforce', 'tag_foreign',
       'tag_aid', 'tag_dod', 'tag_other'],
      dtype='object')

In [90]:
dos_dedupe=dos.groupby(['Title','congress_num']).agg({'Legislation Number': ', '.join,
                                           'URL': 'first', 
                                           'Date of Introduction':'first',
                                           'Latest Action Date': 'last',
                                           'Sponsor': ', '.join,
                                           'Number of Cosponsors': 'max',
                                           'Committees': 'first',
                                           'allsubjects': 'first',
                                           'summary': 'first',
                                           'passed?': 'max',
                                           'relevant_sophia': 'max',
                                           'tag_reform': 'max',
                                           'tag_financial': 'max',
                                           'financial type': 'first',
                                           'tag_workforce': 'max',
                                           'tag_foreign': 'max',
                                           'tag_aid': 'max',
                                           'tag_dod': 'max',
                                           'tag_other': 'max'
                                           }).reset_index()

In [92]:
dos_dedupe.shape

(471, 21)

In [500]:
dos_dedupe.to_csv('dos_dedupe.csv')